No need to run all

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
# exclude all cancelled orders
# only keep the US orders
# exclude all refunded orders ## update on 06/24/2024: keep all refunded orders
# exclude all shopify draft orders
# only keep the non-wholesale orders
# ## update on 07/02/2024: exclude all meta orders
# exclude all empty email orders
# only keep the orders with sku
# exclude all zero pre_tax_price products
# exclude all zero netproducts
# save output in a variable `df`
## 2023-2024 Data

%%bigquery df --project fb-business-ws
SELECT order_number, created_at, shipping_address_province_code, email, source_channel, sku, title, quantity, revenue, net
FROM fb-business-ws.enriched_marketing.shopify_orderlines
WHERE EXTRACT(year from processed_at) IN (2023,2024) AND cancelled_at IS NULL AND shipping_address_country_code = 'US' AND
source_name != 'shopify_draft_order' AND wholesale_tag IS NULL AND meta_tag is NULL AND email != '' AND sku is not null AND pre_tax_price > 0 AND net > 0

Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
############ FUNCTION #############

def max_interval(series):
  if series.isnull().any():
    return pd.NA
  else:
    return series.max()

def clean_orderlines(df):
  """
    Combine non-US province code into 'Other';
    calculate purchasing intervals;
    combine 'source_channel' into several main categories

    Parameters:
    df: The dataframe retrived from BigQuery.

    Returns:
    df: The preprocessed datafrme.
    """

  ## drop duplications
  df = df.drop_duplicates()

  ## combine non-US province code into 'Other'
  USstates = ['AL','AK','AZ','AR','AS','AE','AP','AA','CA','CO','CT','DE','DC','FL','GA','GU','HI','ID','IL','IN','IA','KS','KY','LA',
              'ME','MD','MA','MI','MN','MS','MO','MT','MP','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK',
              'OR','PA','PR','RI','SC','SD','TN','TX','UT','UM','VT','VI','VA','WA','WV','WI','WY']
  df.loc[~df['shipping_address_province_code'].isin(USstates), 'shipping_address_province_code'] = 'Other'

  ## calculate purchasing intervals (number of days between two orders made with the same customer email)
  # sort the dataset by 'email' and 'created_at' to ensure consecutive purchases for each customer
  df = df.sort_values(by=['email', 'created_at'])
  # calculate the interval between consecutive purchases for each customer
  df['interval'] = df.groupby('email')['created_at'].diff().dt.days
  # for rows where 'email' changes, reset the interval to NaN to indicate a new customer
  df['interval'] = df['interval'].where(df['email'].eq(df['email'].shift()))
  # create a dictionary to map 'order_number' to the interval value
  order_id_interval_map = df.groupby('order_number')['interval'].agg(max_interval).to_dict()
  # map the interval value back to each row based on 'order_number'
  df['interval'] = df['order_number'].map(order_id_interval_map)

  ## combine 'source_channel' into several main categories
  df.loc[pd.isnull(df['source_channel']), 'source_channel'] = 'Unkown'
  df.loc[df['source_channel'] == '', 'source_channel'] = 'Unkown'
  # Define mappings of patterns to replacement values
  mapping = {
    'fb|facebook|meta|facbook|facebad': 'Facebook(PS)',
    'ig|instagram': 'Instagram(Other)',
    'bing': 'Bing(PSH)',
    'klaviyo': 'Klaviyo(E)',
    'attentive': 'Attentive(T)',
    # main channels
    'paid|pinterest|youtube|tiktok': 'Paid Social',
    'dailycampaign|text|flow': 'Text',
    'orderlyemail|yotpo|mail|newsletter': 'Email',
    'hoobe|organic|navigation|linkinbio': 'Organic & Social',
    'google|microsoft': 'Paid Search',
    'homepage': 'Display',
    'affluencer|affiliate': 'Affiliate',
    'BeefeylFzbvyf|bethany|ciolota|chris|chrome|google_shopping|klarna|loyalty|narrativ|octoly|refer|shop_app|thingtesting.com|trustpilot|color wow|direct|excluded|tbbtyf|non-attributed|baavyvbgf': 'Other'
  }

  # Use regex to match patterns and replace values in 'main_channel' column
  for pattern, replacement in mapping.items():
    df.loc[df['source_channel'].str.contains(pattern, case=False), 'source_channel'] = replacement
  df.loc[df['source_channel'] == 'social', 'source_channel'] = 'Organic & Social'
  category = ['Unkown','Facebook(PS)','Instagram(Other)','Bing(PSH)','Klaviyo(E)','Attentive(T)','Paid Social','Text','Email','Organic & Social','Paid Search','Display','Affiliate']
  df.loc[~df['source_channel'].isin(category), 'source_channel'] = 'Other'

  ## Resetting the index of the cleaned dataset
  df.reset_index(drop=True, inplace=True)

  return df

In [5]:
df = clean_orderlines(df)
print(df.shape[0])

1460502


In [ ]:
project_id = 'fb-business-datasets'  # Google Cloud project ID
destination_table = 'fb-business-datasets.ds_tables.cleaned_shopify_orderlines_newest'  # destination table name in BigQuery
df.to_gbq(destination_table, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 630.44it/s]


Function Development

In [4]:
# exclude all cancelled orders
# only keep the US orders
# exclude all refunded orders  ## update on 6/24; keep all refunded orders
# exclude all shopify draft orders
# only keep the non-wholesale orders
# exclude all empty email orders
# only keep the orders with sku
# exclude all zero pre_tax_price products
# exclude all zero netproducts
# save output in a variable `df`
## 2023 Data Only

%%bigquery df --project fb-business-ws
SELECT order_number, created_at, shipping_address_province_code, email, source_channel, sku, title, quantity, revenue, net
FROM fb-business-ws.enriched_marketing.shopify_orderlines
WHERE EXTRACT(year from processed_at) = 2023 AND cancelled_at IS NULL AND shipping_address_country_code = 'US' AND source_name != 'shopify_draft_order' AND email != '' AND wholesale_tag is null AND sku is not null AND pre_tax_price > 0 AND net > 0

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df.head()

,order_number,created_at,shipping_address_province_code,email,source_channel,sku,title,quantity,revenue,net
0,1486930,2023-05-18 05:56:10+00:00,TX,irmahinojosa95@gmail.com,klarna,CW419,Easy Blo,1,49.0,49.0
1,1381635,2023-02-27 03:43:50+00:00,GA,maso916@yahoo.com,None,CW419,Easy Blo,1,49.0,49.0
2,1445294,2023-04-13 00:35:22+00:00,CA,shradha.k9@gmail.com,None,CW419,Easy Blo,1,49.0,49.0
3,1669240,2023-08-31 06:11:12+00:00,TX,cindytejeda@verizon.net,daily-klaviyo,CW419,Easy Blo,1,49.0,49.0
4,1374640,2023-02-21 06:41:31+00:00,CA,shannonnew10@gmail.com,None,CW415,Thats a Wrap!,1,22.0,22.0


#### Preprocessing Dataset   
1. drop duplications
2. `shipping_address_province_code`: if not US states than 'Other'
3. `interval`: create such a column to calculate the purchase interval
4. `source_channel`: keep the main channels
5. check if other columns containing null values or not
6. check data types and convert into wanted data types



In [13]:
df.shape

(1643327, 10)

In [14]:
df = df.drop_duplicates()
df.shape

(1643278, 10)

In [15]:
df.isna().sum()

order_number                           0
created_at                             0
shipping_address_province_code         0
email                                  0
source_channel                    684632
sku                                    0
title                                  0
quantity                               0
revenue                                0
net                                    0
dtype: int64

In [16]:
## combine non-US province code into 'Other'
USstates = ['AL','AK','AZ','AR','AS','AE','AP','AA','CA','CO','CT','DE','DC','FL','GA','GU','HI','ID','IL','IN','IA','KS','KY','LA',
            'ME','MD','MA','MI','MN','MS','MO','MT','MP','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK',
            'OR','PA','PR','RI','SC','SD','TN','TX','UT','UM','VT','VI','VA','WA','WV','WI','WY']

df.loc[~df['shipping_address_province_code'].isin(USstates), 'shipping_address_province_code'] = 'Other'
print(df['shipping_address_province_code'].nunique())
print(df['shipping_address_province_code'].unique())

58
['FL' 'NY' 'PA' 'MO' 'DE' 'CO' 'TX' 'CA' 'NE' 'VA' 'NC' 'MN' 'MA' 'OH'
 'IN' 'KY' 'NJ' 'AL' 'WA' 'WI' 'OK' 'WY' 'GA' 'WV' 'TN' 'IL' 'AZ' 'ID'
 'UT' 'HI' 'MI' 'LA' 'NH' 'MS' 'ME' 'SC' 'RI' 'AR' 'KS' 'NV' 'CT' 'MD'
 'IA' 'NM' 'OR' 'MT' 'DC' 'SD' 'ND' 'AK' 'PR' 'VT' 'AE' 'GU' 'VI' 'AA'
 'AP' 'MP']


In [17]:
df['created_at'].min()

Timestamp('2023-01-01 00:01:31+0000', tz='UTC')

In [18]:
## calculate purchasing intervals (number of days between two orders made with the same customer email)

# sort the dataset by 'email' and 'created_at' to ensure consecutive purchases for each customer
df = df.sort_values(by=['email', 'created_at'])
# calculate the interval between consecutive purchases for each customer
df['interval'] = df.groupby('email')['created_at'].diff().dt.days
# for rows where 'email' changes, reset the interval to NaN to indicate a new customer
df['interval'] = df['interval'].where(df['email'].eq(df['email'].shift()))
# Create a dictionary to map 'order_number' to the interval value
def max_interval(series):
    if series.isnull().any():
        return pd.NA
    else:
        return series.max()
order_id_interval_map = df.groupby('order_number')['interval'].agg(max_interval).to_dict()
# Map the interval value back to each row based on 'order_number'
df['interval'] = df['order_number'].map(order_id_interval_map)

In [ ]:
df.head(5)

,order_number,created_at,shipping_address_province_code,email,source_channel,sku,title,quantity,revenue,net,interval
658576,1618165,2023-08-09 20:05:18+00:00,MA,+1508596smbak55@gmail.com,None,CW548,Dream Filter For Picture-Perfect Color,1,24.0,24.0,NaN
281130,1447811,2023-04-15 17:21:29+00:00,FL,0-orange-braces@icloud.com,daily-klaviyo,CW510,Color Security Shampoo,1,24.0,24.0,NaN
613259,1447811,2023-04-15 17:21:29+00:00,FL,0-orange-braces@icloud.com,daily-klaviyo,CW528,Style on Steroids ~ Color-Safe Texturizing Spray,1,26.0,26.0,NaN
101295,1852531,2023-12-18 22:42:37+00:00,TX,0.aliases.iambic@icloud.com,facebook,CW528,Style on Steroids ~ Color-Safe Texturizing Spray,1,26.0,26.0,NaN
552217,1852531,2023-12-18 22:42:37+00:00,TX,0.aliases.iambic@icloud.com,facebook,CWB103,Extra Mist-ical Shine Spray + FREE* Puffer Bag...,1,29.0,29.0,NaN


In [19]:
## clean the source_channel
print(df['source_channel'].nunique())
print(df['source_channel'].unique())
print(df['source_channel'].value_counts(normalize=True) * 100)

224
[None 'daily-klaviyo' 'facebook' 'flow-klaviyo' 'flow-text'
 'dailycampaign-text-' 'dailycampaign-text  ' 'pinterest' 'hoobe'
 'affiliate' 'homepage' 'attentive' 'Facebook' 'tiktok' 'bing'
 'flow-text  ' 'OrderlyEmails' 'Klaviyo' 'yotpo' 'dailycampaign-text'
 'klarna' 'fb' 'navigation' 'Pinterest' 'IGShopping' 'shop_app' ''
 'listrak' 'loyalty' 'paid' 'fbn' 'google' 'dailycampaign-text '
 'PaidSocial' 'youtube' 'SMSBump'
 'dailycampaign-text  utm_source=dailycampaign-text  '
 'Malomo Post Purchase: 4. Delivered' 'Facebook_Mobile_Feed' 'ig'
 'Meta-SiteLink-644-2' 'drip' 'Google' 'Meta-SiteLink-1126-0'
 'B TRACK—Lightly Engaged (30-60 days)' 'Meta-SiteLink-3427-2' 'sfmc'
 'Iterable' 'active_campaign' 'Meta-SiteLink-644-1' 'email'
 'Meta-SiteLink-1133-0' 'fbig' 'CTC FB' 'organic' 'powereditor'
 'linkinbio' 'Meta-SiteLink-190-0' 'USPatriotFlags by Ultimate Flags LLCb'
 'preflect' 'Proozy' 'Digestive Support Protein Customers'
 'MMP-SiteLink-189-0' '6259466001825' 'Meta-SiteLink-188-0' 

In [20]:
df.loc[pd.isnull(df['source_channel']), 'source_channel'] = 'Unkown'
df.loc[df['source_channel'] == '', 'source_channel'] = 'Unkown'
print(df['source_channel'].value_counts(normalize=True) * 100)

source_channel
Unkown                            41.685278
facebook                          26.304679
daily-klaviyo                     11.971985
dailycampaign-text                 3.788403
flow-text                          3.674546
                                    ...    
HappyWax - Master list Segment     0.000061
Q4 - First Touch                   0.000061
NP   Active On Site                0.000061
NM - JC - Waiting for Wows         0.000061
tiktokad                           0.000061
Name: proportion, Length: 224, dtype: float64


In [21]:
print(df['source_channel'].sort_values().unique())

['01 | Wonderment | Shipping | Shipment Created' '23854884683410564'
 '24 months engaged segment' '30 days engage' '360SWEATER HQ' '3829940'
 '6259466001825' '6261289086425' '6290122498225'
 'ALL Past Purchasers - 1X' 'Abandoned Cart - NEW YEARS SALE 2023'
 'Active Customers   Active on Site'
 'Active Customers   Active on site addition' 'All B2C Subscribers'
 'All ESI Subscribers' 'All Engaged 45 Day'
 'B TRACK—Lightly Engaged (30-60 days)'
 'Bagnet SMS Subscribers - Consented for SMS' 'Bar Buyers'
 'Beadaholique Insider' 'Bees Wrap Master List' 'Black Friday List'
 'Brand Enthusiasts'
 'CP Loyalty Master List_ALL SUBSCRIBERS _180 Engaged 81121' 'CTC FB'
 'Campaigns: Smaller Send' 'Chris' 'Digestive Support Protein Customers'
 'EMAIL CAMPAIGNS' 'Email' 'Engaged 3 Months (Exclude MPP Openers)' 'FB'
 'FX - Retention Full Engagement - 60 ' 'Facebook' 'FacebookADS'
 'Facebook_Ads fb' 'Facebook_Mobile_Feed' 'GO' 'GOOGLE' 'Google'
 'Growth Gurus | Omnilux LED | Engaged Contacts 120 Days'
 '

In [22]:
channel = pd.DataFrame()
channel['source_channel'] = df['source_channel'].unique()
channel.to_csv('channel.csv')

In [23]:
# Define mappings of patterns to replacement values
mapping = {
    'fb|facebook|meta|facbook|facebad': 'Facebook(PS)',
    'ig|instagram': 'Instagram(Other)',
    'bing': 'Bing(PSH)',
    'klaviyo': 'Klaviyo(E)',
    'attentive': 'Attentive(T)',
    # main channels
    'paid|pinterest|youtube|tiktok': 'Paid Social',
    'dailycampaign|text|flow': 'Text',
    'orderlyemail|yotpo|mail|newsletter': 'Email',
    'hoobe|organic|navigation|linkinbio': 'Organic & Social',
    'google|microsoft': 'Paid Search',
    'homepage': 'Display',
    'affluencer|affiliate': 'Affiliate',
    'BeefeylFzbvyf|bethany|ciolota|chris|chrome|google_shopping|klarna|loyalty|narrativ|octoly|refer|shop_app|thingtesting.com|trustpilot|color wow|direct|excluded|tbbtyf|non-attributed|baavyvbgf': 'Other'
}

# Use regex to match patterns and replace values in 'main_channel' column
for pattern, replacement in mapping.items():
    df.loc[df['source_channel'].str.contains(pattern, case=False), 'source_channel'] = replacement
df.loc[df['source_channel'] == 'social', 'source_channel'] = 'Organic & Social'

In [24]:
category = ['Unkown','Facebook(PS)','Instagram(Other)','Bing(PSH)','Klaviyo(E)','Attentive(T)','Paid Social','Text','Email','Organic & Social','Paid Search','Display','Affiliate']
df.loc[~df['source_channel'].isin(category), 'source_channel'] = 'Other'

In [25]:
print(df['source_channel'].value_counts(normalize=True).head(10) * 100)
print(df['source_channel'].unique())

source_channel
Unkown              41.685278
Facebook(PS)        26.410078
Klaviyo(E)          14.335980
Instagram(Other)     5.968497
Text                 3.725480
Paid Social          3.076716
Attentive(T)         2.305940
Bing(PSH)            0.762257
Affiliate            0.554745
Organic & Social     0.539775
Name: proportion, dtype: float64
['Unkown' 'Klaviyo(E)' 'Facebook(PS)' 'Text' 'Instagram(Other)'
 'Paid Social' 'Organic & Social' 'Affiliate' 'Display' 'Attentive(T)'
 'Bing(PSH)' 'Email' 'Other' 'Paid Search']


In [ ]:
######## filter out wholesale orders

# wholesale = df[['email', 'order_number', 'created_at', 'interval']]
# wholesale = wholesale[~pd.isna(wholesale['interval'])]
# wholesale = wholesale.drop_duplicates()
# wholesale.head()

In [ ]:
# wholesale[wholesale['email'] == 'carlal@plano.gov']

In [ ]:
# print(df.shape)
# print(wholesale.shape)

In [ ]:
# wholesale = wholesale.sort_values(by=['email', 'created_at'])
# wholesale_ids = wholesale.groupby('email').filter(lambda x: (x['interval'] == 0).any())
# wholesale_ids

In [ ]:
# from tqdm import tqdm

# # Define a function to filter groups
# def filter_group(group):
#     # Your filtering logic here
#     return group if (group['interval'] == 0).any() else None

# # Create an empty list to store filtered DataFrames
# filtered_dfs = []

# # Get unique groups
# groups = wholesale.groupby('email')

# # Create a tqdm progress bar
# with tqdm(total=len(groups)) as pbar:
#     # Iterate over groups and filter them
#     for name, group in groups:
#         filtered_df = filter_group(group)
#         if filtered_df is not None:
#             filtered_dfs.append(filtered_df)
#         pbar.update(1)

# # Concatenate the filtered DataFrames
# wholesale_ids = pd.concat(filtered_dfs)

In [ ]:
# wholesale_id = wholesale_ids['email'].unique()
# print(wholesale.shape)
# nonwholesale = wholesale[~wholesale['email'].isin(wholesale_id)]
# print(nonwholesale.shape)
# print(nonwholesale['email'].nunique())

In [ ]:
# filtered_customer_ids = df.groupby('customer_id').filter(lambda x: (x['interval'] <= 1).sum() >= 2)

In [ ]:
df.head()

,order_number,created_at,shipping_address_province_code,email,source_channel,sku,title,quantity,revenue,net,interval
658576,1618165,2023-08-09 20:05:18+00:00,MA,+1508596smbak55@gmail.com,Unkown,CW548,Dream Filter For Picture-Perfect Color,1,24.0,24.0,NaN
281130,1447811,2023-04-15 17:21:29+00:00,FL,0-orange-braces@icloud.com,Klaviyo(E),CW510,Color Security Shampoo,1,24.0,24.0,NaN
613259,1447811,2023-04-15 17:21:29+00:00,FL,0-orange-braces@icloud.com,Klaviyo(E),CW528,Style on Steroids ~ Color-Safe Texturizing Spray,1,26.0,26.0,NaN
101295,1852531,2023-12-18 22:42:37+00:00,TX,0.aliases.iambic@icloud.com,Facebook(PS),CW528,Style on Steroids ~ Color-Safe Texturizing Spray,1,26.0,26.0,NaN
552217,1852531,2023-12-18 22:42:37+00:00,TX,0.aliases.iambic@icloud.com,Facebook(PS),CWB103,Extra Mist-ical Shine Spray + FREE* Puffer Bag...,1,29.0,29.0,NaN


In [26]:
df.isna().sum()

order_number                            0
created_at                              0
shipping_address_province_code          0
email                                   0
source_channel                          0
sku                                     0
title                                   0
quantity                                0
revenue                                 0
net                                     0
interval                          1176670
dtype: int64

In [ ]:
df.dtypes

order_number                                    Int64
created_at                        datetime64[us, UTC]
shipping_address_province_code                 object
email                                          object
source_channel                                 object
sku                                            object
title                                          object
quantity                                        Int64
revenue                                       float64
net                                           float64
interval                                      float64
dtype: object

In [ ]:
df.shape

(940368, 11)